In [ ]:
!git clone https://github.com/weharris/birdnetTest

In [ ]:
cd birdnetTest/

In [ ]:
!pip3 install birdnetlib
!pip3 install tflite-runtime
!pip3 install resampy
!pip3 install ffmpeg

In [1]:
%run scripts/dataFiles.py
print(len(file_names), " files")
print(file_names)


9  files
['BAT01_20240420_080000.wav', 'BAT01_20240420_090000.wav', 'BAT01_20240420_100000.wav', 'BAT01_20240420_110000.wav', 'BAT01_20240420_120000.wav', 'BAT01_20240420_130000.wav', 'BAT01_20240420_140000.wav', 'BAT01_20240420_150000.wav', 'BAT01_20240420_160000.wav']


In [ ]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime

In [ ]:
# This celluses BirdNET-Analyzer to analyze a single recording.
# Load and initialize the BirdNET-Analyzer models.
analyzer = Analyzer()

testamp = [52.911,-2.4441] # near Market Drayton year=2024, month=4, day=5
xc558223 = [57.4706, -4.163] # Highlands Scotland 2020-05-14 4am year=2020, month=5, day=14
XC553677 = [54.1212, -0.5416] # Ryedale District,(near Helperthorpe) North Yorkshire 2020-05-01 5:10a year=2020, month=5, day=1

recording = Recording(
    analyzer,
    "testamp.mp3",  # path to the recording file
    lat=testamp[0], # latitude of the recording location
    lon=testamp[1], # longitude of the recording location
    date=datetime(year=2024, month=4, day=20), # use date or week_48
    min_conf=0.25,
)
recording.analyze()

In [ ]:
# This cell uses BirdNET-Analyzer to analyze multiple recordings.
# Load and initialize the BirdNET-Analyzer models.

analyzer = Analyzer()

testamp = [52.911,-2.4441] # near Market Drayton year=2024, month=4, day=5

# Loop through all file names
for file_name in file_names:
    # Include the correct path to the data files
    file_path = os.path.join('data', file_name)
    recording = Recording(
        analyzer,
        file_path,  # path to the recording file
        lat=testamp[0], # latitude of the recording location
        lon=testamp[1], # longitude of the recording location
        date=datetime(year=2024, month=4, day=20), # use date or week_48
        min_conf=0.25,
    )
    recording.analyze()


In [ ]:
testamp = recording.detections
print(testamp)

In [ ]:
testamp[0]

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
species_count = {}
for entry in testamp:
    common_name = entry['common_name']
    if common_name in species_count:
        species_count[common_name] += 1
    else:
        species_count[common_name] = 1

df = pd.DataFrame(list(species_count.items()), columns=['Common Name', 'Count'])

In [ ]:
# print(df)

sorted_df = df.sort_values('Common Name')
print(sorted_df)

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))  # Set the figure size
plt.bar(df['Common Name'], df['Count'], color='skyblue')  # Create a bar plot
plt.xlabel('Species')  # X-axis label
plt.ylabel('Count')  # Y-axis label
plt.title('Count of Bird Observations by Species')  # Title of the plot
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()  # Display the plot

In [ ]:
analyzer = Analyzer()

recording = Recording(
    analyzer,
    "test.mp3",
    lat=52.911,
    lon=-2.4441,
    date=datetime(year=2024, month=4, day=5), # use date or week_48
    min_conf=0.25,
)
recording.analyze()

In [ ]:
test = recording.detections
print(test)

In [ ]:
# Extract confidence values for "Great Tit"
confidence_testamp = [entry['confidence'] for entry in testamp if entry['common_name'] == 'Great Tit']
confidence_test = [entry['confidence'] for entry in test if entry['common_name'] == 'Great Tit']

# Create DataFrame for plotting
data = {
    "testamp": confidence_testamp,
    "test": confidence_test
}
df = pd.DataFrame(data)

# Plotting
plt.figure(figsize=(8, 6))
df.boxplot(column=['testamp', 'test'])
plt.ylabel('Confidence')
plt.title('Confidence Levels for Great Tit Observations')
plt.show()



In [ ]:
# Perform a t-test
from scipy.stats import ttest_ind
t_stat, p_value = ttest_ind(confidence_testamp, confidence_test, equal_var=False)
print(f"T-statistic: {t_stat}, P-value: {p_value}")

## Sources
https://github.com/kahst/BirdNET-Analyzer?tab=readme-ov-file#62-setup-birdnetlib